# MongoDB Field Level Encryption Demo

In [183]:
import pymongo
from pymongo import MongoClient
from pymongo.encryption import ClientEncryption
from bson import Binary
import pymongocrypt
import base64
from bson.codec_options import DEFAULT_CODEC_OPTIONS

### General Settings:

Generate CMK Example Shell Command: head -c 96 /dev/urandom | base64 > key.txt

In [184]:
cmk = base64.b64decode("IkhJDuh1kQXt7cNlrBkKEMpV/jbpUERloqPGCB0WaNhFC7Ig4z1X2dDwv6XByMlrRyadd3MEC+BdcY8D9cadCgvHe3MVa8o1m00+xcE4KDFJaID/FS1rwZKRPK3KfezP")

In [185]:
database = 'hr'
keyStore = 'dataKeys'
kms_providers = { "local" : { "key" : cmk } }

### Initialise Key Vault:

In [186]:
keyVaultClient = pymongo.MongoClient("127.0.0.1")

In [187]:
key_vault_namespace = database + '.' + keyStore

In [188]:
keyVault = ClientEncryption(kms_providers, key_vault_namespace, keyVaultClient, DEFAULT_CODEC_OPTIONS)

### Create Data Key

Create two data encryption keys:

In [189]:
keyVault.create_data_key("local", master_key=cmk, key_alt_names=["key1"])

Binary(b'L\xbe(\xda\xf3\xe7O\x18\x92_\xb6U\xfd\xfc\x0c\xcd', 4)

In [190]:
keyVault.create_data_key("local", master_key=cmk, key_alt_names=["key2"])

Binary(b'd\x87\xed\xae\x8c\x07Er\x81\xd6\xb0y,\x15\x15\xc4', 4)

Retrieve the key _id to be referenced in the schema_map:

In [191]:
keyDb = keyVaultClient.hr
key1 = keyDb['dataKeys'].find_one({'keyAltNames':'key1'})['_id']
key2 = keyDb['dataKeys'].find_one({'keyAltNames':'key2'})['_id']

Define the encryption schema:

In [192]:
schema_map = {
    "hr.employees":
    {
        'bsonType': 'object',
        'encryptMetadata': {
            'keyId': [key1],
            'algorithm': 'AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic'
            },
        'properties': {
            'ssn': {
                'encrypt': {
                    'bsonType': 'string'
                    }
                },
            'position': {
                'bsonType': 'object',
                'encryptMetadata': {
                    'keyId': [key2],
                    'algorithm': 'AEAD_AES_256_CBC_HMAC_SHA_512-Random'
                    },
                'properties': {
                    'compensation': {
                        'encrypt': {}
                        }
                }}
            }
        }
    }

Configure the field level encryption client:

In [193]:
fleOptions = pymongo.encryption_options.AutoEncryptionOpts(kms_providers, key_vault_namespace, key_vault_client=None, schema_map=schema_map, bypass_auto_encryption=False, mongocryptd_uri='mongodb://localhost:27020', mongocryptd_bypass_spawn=False, mongocryptd_spawn_path='mongocryptd', mongocryptd_spawn_args=None)

Initialise the secured client:

In [194]:
secClient = pymongo.MongoClient("127.0.0.1",auto_encryption_opts = fleOptions)

### Insert Data

In [195]:
employee = {
    'name': 'John Doe',
    'address': {'street': '1234 Main Street',
    'city': 'MongoDBVille',
    'zip': 99999},
    'phone': '949-555-1212',
    'ssn': '123-45-6789',
    'position': {'compensation': 1234}
}

In [196]:
db = secClient.hr
db.employees.insert_one(employee)

### Read Data

In [197]:
db = secClient.hr
db.employees.find_one({'ssn': '123-45-6789'})

{'_id': ObjectId('5da6021e2158d2f8477d0154'),
 'name': 'John Doe',
 'address': {'street': '1234 Main Street',
  'city': 'MongoDBVille',
  'zip': 99999},
 'phone': '949-555-1212',
 'ssn': '123-45-6789',
 'position': {'compensation': 1234}}

### Use Compass or the following to show the encrypted values

In [198]:
unsecClient = pymongo.MongoClient("127.0.0.1")
db = unsecClient.hr
db.employees.find_one({'ssn': '123-45-6789'})

In [199]:
db.employees.find_one({'name': 'John Doe'})

{'_id': ObjectId('5da6021e2158d2f8477d0154'),
 'name': 'John Doe',
 'address': {'street': '1234 Main Street',
  'city': 'MongoDBVille',
  'zip': 99999},
 'phone': '949-555-1212',
 'ssn': Binary(b'\x01L\xbe(\xda\xf3\xe7O\x18\x92_\xb6U\xfd\xfc\x0c\xcd\x02H\xda\xd5\xf13\x1f\xf4\xb2\xa8\x02C\x04\xe3\xfa\xf9{\x98\xe9\xbd\x0c-?\xb9 \xc4\xc5\xeaM\xdb\x9e\xc3\xd7~|G\x0b\xbd\xd1\x18\xe7\xa4\x8d\xd2b\x10u\xca\x90j\xcd\x1c"X\xa5J\x1aI\xb2\xf9\xf1\x8cW_>\xf1\x0f\x8eP\x8d\rU\x87_l\xb1\x8e\xd18g$', 6),
 'position': {'compensation': Binary(b'\x02d\x87\xed\xae\x8c\x07Er\x81\xd6\xb0y,\x15\x15\xc4\x10U\xbf\xf5b\x05\x90\x11}\xefY\x84\x8b\x8e~m\xc7\xabZ\x83<\x12\xe3alm\xcc\x15\xa7T\xf9k\xce\x80>I\x0b\xd5\x00\xdeL\xb9\n\xff#\x82+\xf2\xe1,\xdb\x93(k\xf0\xe2\x05Y\x9e\xef\xa7,\xfa\xe8\xf0', 6)}}